In [8]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.cluster import MiniBatchKMeans
import re
from collections import Counter

In [11]:
data = pd.read_csv('validation.csv.zip')

In [13]:
matches = []
for t,l in data[['comment_text', 'lang']].values:
    m = re.findall('\w*\*{2,10}\w*', t)
    
    if m:
        matches += [(ma, l) for ma in m]

In [14]:
Counter(matches).most_common()

[(('f**k', 'es'), 11),
 (('***İyi', 'tr'), 4),
 (('vikilemeler***', 'tr'), 4),
 (('ca**i', 'it'), 3),
 (('***', 'tr'), 3),
 (('***Osmanlı98', 'tr'), 3),
 (('c***o', 'it'), 2),
 (('p***', 'es'), 2),
 (('M*****', 'it'), 2),
 (('****', 'es'), 2),
 (('ca**o', 'it'), 2),
 (('i****zzerà', 'it'), 1),
 (('ca***ta', 'it'), 1),
 (('P***', 'es'), 1),
 (('**', 'tr'), 1),
 (('ca***e', 'it'), 1),
 (('C***O', 'it'), 1),
 (('M*****A', 'it'), 1),
 (('co**nes', 'es'), 1),
 (('*****', 'it'), 1),
 (('pin**', 'es'), 1),
 (('m***a', 'it'), 1),
 (('quiiii******', 'it'), 1),
 (('f*****o', 'it'), 1),
 (('h**vos', 'es'), 1),
 (('M****', 'it'), 1),
 (('PA***', 'es'), 1),
 (('c****', 'it'), 1),
 (('p***e', 'it'), 1),
 (('M***A', 'it'), 1),
 (('ca**ta', 'it'), 1),
 (('ca***te', 'it'), 1),
 (('pu**', 'es'), 1),
 (('pinc**', 'es'), 1),
 (('mier**', 'es'), 1),
 (('p***ta', 'es'), 1),
 (('destacadísimo*****', 'es'), 1),
 (('ca***', 'it'), 1),
 (('inca**to', 'it'), 1),
 (('mie***da', 'es'), 1),
 (('c***', 'it'), 1),
 (

In [9]:
texts = data[data.lang=='ru']['content'].values

In [10]:
from sklearn.cluster import AffinityPropagation, AgglomerativeClustering, DBSCAN, \
                            KMeans, MiniBatchKMeans, Birch, MeanShift, SpectralClustering
from sklearn.metrics import adjusted_rand_score, adjusted_mutual_info_score, \
                            silhouette_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD, NMF
from sklearn import metrics
from collections import defaultdict

In [12]:
cv = CountVectorizer(max_features=10000,ngram_range=(1,3),analyzer='char_wb', min_df=5, max_df=0.3)
X = cv.fit_transform(texts)
svd = TruncatedSVD(100)
X_svd = svd.fit_transform(X)

In [14]:
cluster = MiniBatchKMeans(500, n_init=2, verbose=1, batch_size=500,
                          max_no_improvement=20, reassignment_ratio=0.4)
cluster.fit(X)

Init 1/2 with method: k-means++
Inertia for init 1/2: 484849.504536
Init 2/2 with method: k-means++
Inertia for init 2/2: 496452.322527
Minibatch iteration 1/2200: mean batch inertia: 497.243681, ewa inertia: 497.243681 
Minibatch iteration 2/2200: mean batch inertia: 424.350316, ewa inertia: 490.586145 
Minibatch iteration 3/2200: mean batch inertia: 409.833915, ewa inertia: 483.210839 
Minibatch iteration 4/2200: mean batch inertia: 422.167588, ewa inertia: 477.635603 
Minibatch iteration 5/2200: mean batch inertia: 409.636626, ewa inertia: 471.425084 
Minibatch iteration 6/2200: mean batch inertia: 600.042811, ewa inertia: 483.172068 
Minibatch iteration 7/2200: mean batch inertia: 359.725860, ewa inertia: 471.897412 
Minibatch iteration 8/2200: mean batch inertia: 375.300038, ewa inertia: 463.074928 
Minibatch iteration 9/2200: mean batch inertia: 423.967744, ewa inertia: 459.503170 
[MiniBatchKMeans] Reassigning 250 cluster centers.
Minibatch iteration 10/2200: mean batch inertia:

MiniBatchKMeans(batch_size=500, compute_labels=True, init='k-means++',
        init_size=None, max_iter=100, max_no_improvement=20,
        n_clusters=500, n_init=2, random_state=None,
        reassignment_ratio=0.4, tol=0.0, verbose=1)

In [15]:
labels = cluster.labels_

In [16]:
n_clusters = len(set(labels))

In [17]:
# cluster = DBSCAN(min_samples=2, eps=0.9, leaf_size=5) 
# cluster.fit(X_svd)

In [18]:
# print("Silhouette Coefficient: %0.3f"
#       % metrics.silhouette_score(X, labels))

In [20]:
cls = defaultdict(list)
for i, cl in enumerate(cluster.labels_):
    cls[cl].append(texts[i])

# откройте в любом текстовом редакторе
f = open('clusters.txt', 'w')
for cl in cls:
    f.write('### '+ str(cl) + ' ###\n')
    f.write('\n'.join(cls[cl]))
    f.write('\n\n')
f.close()